In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict

In [2]:
PATH = "/Volumes/BC_Clutch/Dropbox/Programming/algorithms/data/wine/wine.data"

In [3]:
wine_df = pd.read_csv(PATH,header=None)
wine_df.columns = ['wine','alcohol','malic_acid','ash','alcalinity','magnesium',
               'phenols','flavanoids','nonflavanoid_phenols','proanthocyanins',
               'color_intensity','hue','od280','proline']

In [4]:
wine_df

,wine,alcohol,malic_acid,ash,alcalinity,magnesium,phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280,proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.640000,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.380000,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.680000,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.800000,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.320000,1.04,2.93,735
5,1,14.20,1.76,2.45,15.2,112,3.27,3.39,0.34,1.97,6.750000,1.05,2.85,1450
6,1,14.39,1.87,2.45,14.6,96,2.50,2.52,0.30,1.98,5.250000,1.02,3.58,1290
7,1,14.06,2.15,2.61,17.6,121,2.60,2.51,0.31,1.25,5.050000,1.06,3.58,1295
8,1,14.83,1.64,2.17,14.0,97,2.80,2.98,0.29,1.98,5.200000,1.08,2.85,1045
9,1,13.86,1.35,2.27,16.0,98,2.98,3.15,0.22,1.85,7.220000,1.01,3.55,1045


In [12]:
wine_df.groupby("wine")["wine"].count()

wine
1    59
2    71
3    48
Name: wine, dtype: int64

In [41]:
def stratified_sampling(df, cls_col, k):
    class_dict = defaultdict(list)
    fold_idxs = defaultdict(list)

    for i in range(len(df)):
        cls = int(df.iloc[i,:][cls_col])
        class_dict[cls].append(i)
    
    print("Initial Distribution:")
    for i, cls in enumerate(class_dict.keys()):
        print(f"Class {i}: {len(class_dict[cls])}, {100*(len(class_dict[cls])/len(wine_df)):.1f}%")
        
    for cls in class_dict.keys():
        idxs = class_dict[cls]
        i = 0
        while len(idxs)>0:
            idx = np.random.choice(idxs,replace=False)
            popped = idxs.pop(idxs.index(idx))
            fold_idxs[i].append(popped)
            if i == k-1:
                i = 0
            else:
                i+=1
    
    for k in fold_idxs.keys():
        dfk = df.iloc[fold_idxs[k]]
        dfk = dfk.groupby(cls_col)[cls_col].count()
        dfk.columns = ['count']
        dfk['pct'] = dfk['count'].apply(lambda x: x / np.sum(dfk['count']))
        print(dfk)
    
    assert len(set(df.iloc[fold_idxs[k-1]].index))==len(wine_df.iloc[fold_idxs[k-1]].index)
    return fold_idxs

In [42]:
fold_idxs = stratified_sampling(wine_df, "wine",5)

Initial Distribution:
Class 0: 59, 33.1%
Class 1: 71, 39.9%
Class 2: 48, 27.0%


KeyError: 'count'

### Create Dictionary of Indexes by Class

In [20]:
class_dict = defaultdict(list)
fold_idxs = defaultdict(list)

for i in range(len(wine_df)):
    wine = int(wine_df.iloc[i,:]['wine'])
    class_dict[wine].append(i)
class_dict

for cls in class_dict.keys():
    print(f"{100*(len(class_dict[cls])/len(wine_df)):.1f}%")

33.1%
39.9%
27.0%


### Evenly distribute indexes amongst all folds

In [6]:
for cls in class_dict.keys():
    idxs = class_dict[cls]
    i = 0
    while len(idxs)>0:
        idx = np.random.choice(idxs,replace=False)
        popped = idxs.pop(idxs.index(idx))
        fold_idxs[i].append(popped)
        if i == 4:
            i = 0
        else:
            i+=1
fold_idxs

defaultdict(list,
            {0: [52,
              13,
              3,
              45,
              30,
              9,
              46,
              35,
              58,
              32,
              36,
              22,
              60,
              115,
              84,
              68,
              88,
              121,
              110,
              62,
              109,
              63,
              103,
              105,
              129,
              127,
              92,
              152,
              160,
              136,
              168,
              177,
              163,
              153,
              131,
              147,
              150],
             1: [41,
              43,
              8,
              7,
              47,
              54,
              57,
              44,
              31,
              20,
              25,
              27,
              61,
              99,
              66,
              71,
       

In [7]:
for k in fold_idxs.keys():
    print(len(fold_idxs[k]))

37
36
36
35
34


In [8]:
fold_idxs[0]

[52,
 13,
 3,
 45,
 30,
 9,
 46,
 35,
 58,
 32,
 36,
 22,
 60,
 115,
 84,
 68,
 88,
 121,
 110,
 62,
 109,
 63,
 103,
 105,
 129,
 127,
 92,
 152,
 160,
 136,
 168,
 177,
 163,
 153,
 131,
 147,
 150]

In [11]:
wine_df.iloc[fold_idxs[0]].groupby("wine")["wine"].count()

wine
1    12
2    15
3    10
Name: wine, dtype: int64

In [13]:
wine_df.iloc[fold_idxs[1]].groupby("wine")["wine"].count()

wine
1    12
2    14
3    10
Name: wine, dtype: int64

In [14]:
wine_df.iloc[fold_idxs[2]].groupby("wine")["wine"].count()

wine
1    12
2    14
3    10
Name: wine, dtype: int64

In [15]:
wine_df.iloc[fold_idxs[3]].groupby("wine")["wine"].count()

wine
1    12
2    14
3     9
Name: wine, dtype: int64

In [16]:
wine_df.iloc[fold_idxs[4]].groupby("wine")["wine"].count()

wine
1    11
2    14
3     9
Name: wine, dtype: int64

In [19]:
len(set(fold_idxs[0]+fold_idxs[1]+fold_idxs[2]+fold_idxs[3]+fold_idxs[4]))==len(wine_df)

True